In [4]:
import jieba
import pandas as pd
from collections import defaultdict
import math
import operator
from tqdm import tqdm

jieba.initialize()

# 最简单的TF-IDF分类算法

In [2]:
def load_abstract():
    train_df_iter = pd.read_csv('datasets/train_dataset.csv', sep='|')
    results = []
    for i in train_df_iter['abstract'].tolist():

        target = jieba.cut(i, HMM=False)
        results.append([i for i in target])
    return results

def feature_select(list_words):
    # 总词频统计
    doc_frequency = defaultdict(int)
    for word_list in list_words:
        for i in word_list:
            doc_frequency[i] += 1

    # 计算每个词的TF值
    word_tf = {}  # 存储没个词的tf值
    for i in doc_frequency:
        word_tf[i] = doc_frequency[i] / sum(doc_frequency.values())

    # 计算每个词的IDF值
    doc_num = len(list_words)
    word_idf = {}  # 存储每个词的idf值
    word_doc = defaultdict(int)  # 存储包含该词的文档数
    for i in doc_frequency:
        for j in list_words:
            if i in j:
                word_doc[i] += 1
    for i in doc_frequency:
        word_idf[i] = math.log(doc_num / (word_doc[i] + 1))

    # 计算每个词的TF*IDF的值
    word_tf_idf = {}
    for i in doc_frequency:
        word_tf_idf[i] = word_tf[i] * word_idf[i]

    # 对字典按值由大到小排序
    dict_feature_select = sorted(word_tf_idf.items(), key=operator.itemgetter(1), reverse=True)
    return dict_feature_select


data_list = load_abstract()  # 加载数据
features = feature_select(data_list)  # 所有词的TF-IDF值
print(len(features))

with open('datasets/tf_idf.text', 'w', encoding='UTF-8') as writer:
    writer.writelines('\n'.join([f'{i[0]} {i[1]*1000}' for i in features]))

KeyboardInterrupt: 

In [134]:
import jieba.analyse
import pandas as pd
jieba.analyse.set_idf_path("./datasets/tf_idf.text")

def find_key_words(row):
    return jieba.analyse.textrank(row.encode('utf-8'), topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))

train_df['textrank'] = train_df['abstract'].apply(find_key_words)
train_df = train_df.rename(columns={'keys':'high_key'})

train_df = train_df[~(train_df["abstract"].str.contains('\t\t')&train_df["abstract"].str.contains(':'))]
train_df = train_df[~(train_df["abstract"].str.contains('1、')&train_df["abstract"].str.contains(':'))]

def find_key_words(row):
    result = []
    for i in ['限速', '骚扰', '前期', '信号', '短信', '基站', '额度', '流量', '前期', '减免']:
        if i in row['abstract']:
            result.append(i)
    return result

train_df['own_key'] = train_df.apply(find_key_words,axis=1)
train_df.to_csv('datasets/train_dataset_key.csv', index=False, sep='|')

In [ ]:
相似的信息
信用额度：透支
中文转数字
带有'\t'得都删除


限速
骚扰
前期
信号
短信
基站
信用额度
流量
前期
减免

所在位置->基站
#rule base
奇怪的abstract的格式，
交了费用但是没有续费成功


In [5]:
train_df_iter = pd.read_csv('datasets/train_dataset.csv', sep='|', chunksize=1000)
for train_df in tqdm(train_df_iter):
    break

0it [00:00, ?it/s]


In [8]:
cc = train_df['content'].loc[0]

In [10]:
cc.split('【坐席】')

['',
 '您好，实习话务员为您，【客户】服务，喂，你好，是这样的，我那个我的电话因为上次突然把给我停机了，我不知道啥意思，然后我就好长时间没有使用，我这次拿着10，怎么我欠费160多了？噢，我想问一下，我这机器现在是在停机状态吧，',
 '嗯，对是的？【客户】对停机状态，你们怎么还收我费用啊，',
 '嗯，您稍等这边看的话，先生，咱们这12月份的话是没有收您的费用的，这边是这个产生的费用是您11月份的女士先生，【客户】那我11月份我现在-160，那我想问一下，那我这个怎么能-160？我始终没超一个星期，他就你们就给我停机了，因为那我忙着在外地出差，我就没有，我就没有那个办这个找你们，',
 '嗯，先生这边看的话，先生，您是一个就是11月份的话，有一个是，嗯，基本套餐费是88块钱，还有您上个月未结清的话费是76块钱，【客户】但是我刚才你们给我反馈那个是我欠160多，',
 '嗯，对是的先生，就是您这160多，嗯，160多是产生的，是11月份的结转11月份的，总消费是88，还有您10月还有一个是10月份未结清的话，是76块钱？【客户】我没有结清的，我就想问你，你帮我查一下，我打几天电话，我哪来的那个？还有那么多没结清呢，我就不明白，我一个包月电话我都没有打出去多少电话，我这流我这是4000分钟，是两三千分钟啊，那那么多话费我根本都没有打几个，他怎么就我又欠那么多钱呢',
 '嗯，先生，您这个是阿里大包，嗯，就是三角大大宝卡吗？先生每个月基本月租费是88块钱的，里面是给您包含500分钟通话工作，工作员是二百两千五百分钟，相当于3000分钟吗？先生，还有两个G的流量，【客户】对呀，',
 '嗯，它这个是如果您，嗯没有超出的话，每个月是收您88块钱的先生，【客户】对呀，那我也没有超出啊，你怎么还有我查，我这还欠那么多费呢',
 '这嗯，就每个月正常扣您的套餐费用，先生88块钱套餐费用，先生这个是，【客户】对，但是我这个你看一下，你看看你从我开始，我注册这号到停机，您看看我使这几天你你对你们，你查一下看我使了几天，我怎么的就是有这个还有欠费呢，我这还没，我还说实在我找你们，我这使那几天，我那88块钱你们都给我退我，',
 '嗯这边看的话是在9月份给您给您停的机先生，【客户】对啊，9月份我是9月几号开通的这个卡呀',
 '嗯，您这是9月1号给您开通的卡先生，【客户】对9月份9